In [13]:
# Core Modules
import pandas as pd
import numpy as np

# Basic modules for data visualization
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [14]:
test_df = pd.read_csv('clean_test.csv')
train_df = pd.read_csv('clean_train.csv')

In [15]:
train_df.shape

(1421, 71)

In [16]:
test_df.shape

(1459, 70)

In [17]:
final_df = pd.concat([train_df,test_df],axis=0)
final_df.shape

(2880, 71)

In [18]:
def category_onehot_multcols(multcolumns):
    df_final = final_df
    i = 0
    for fields in multcolumns:
        print(fields)
        df1 = pd.get_dummies(final_df[fields],drop_first=True)
        final_df.drop([fields],axis=1,inplace=True)
        if i==0:
            df_final = df1.copy()
        else:
            df_final = pd.concat([df_final,df1],axis=1)
        i = i + 1

    df_final = pd.concat([final_df,df_final],axis = 1)
    return df_final

In [19]:
cat_attributes = final_df.select_dtypes(include='object')

In [20]:
final_df_1hot = final_df

final_df_1hot = category_onehot_multcols(cat_attributes)

MSZoning
Street
Alley
LotShape
LandContour
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
KitchenQual
Functional
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PavedDrive
PoolQC
Fence
MiscFeature
SaleType
SaleCondition


In [21]:
final_df_1hot.shape

(2880, 243)

In [22]:
final_df_1hot = final_df_1hot.loc[:,~final_df_1hot.columns.duplicated()]
final_df_1hot.shape

(2880, 167)

In [23]:
df_train = final_df_1hot.iloc[:1421,:]
df_test = final_df_1hot.iloc[1421:,:]

In [24]:
df_train.shape

(1421, 167)

In [25]:
df_test.shape

(1459, 167)

In [26]:
df_test.drop(['SalePrice'],axis=1,inplace=True)

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [27]:
x_train = df_train.drop(['SalePrice'],axis=1)
y_train = df_train['SalePrice']

### First Model (Score = 0.14549)

In [28]:
from sklearn.ensemble import RandomForestRegressor

rf_model_final = RandomForestRegressor(bootstrap=False,max_features=40, n_estimators=100, random_state=5)

rf_model_final.fit(x_train, y_train)

RandomForestRegressor(bootstrap=False, max_features=40, random_state=5)

In [22]:
rf_model_final_pred = rf_model_final.predict(df_test)

### Hyperparameter Tuning 1 (Score = 0.14541)

In [29]:
from sklearn.model_selection import GridSearchCV
import numpy as np

max_features_range = np.arange(20,65,5)
n_estimators_range = np.arange(80,120,5)
grid_params = {'max_features':max_features_range,'n_estimators':n_estimators_range}
rf = RandomForestRegressor(bootstrap=False,random_state=5)

grid_search = GridSearchCV(estimator = rf, param_grid = grid_params,cv=5,n_jobs=-1,verbose=1)

In [30]:
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(cv=5,
             estimator=RandomForestRegressor(bootstrap=False, random_state=5),
             n_jobs=-1,
             param_grid={'max_features': array([20, 25, 30, 35, 40, 45, 50, 55, 60]),
                         'n_estimators': array([ 80,  85,  90,  95, 100, 105, 110, 115])},
             verbose=1)

In [31]:
print(grid_search.best_params_)
print(grid_search.best_score_)

{'max_features': 35, 'n_estimators': 95}
0.9044553795637252


In [32]:
rf = RandomForestRegressor(bootstrap=False,max_features=35, n_estimators=95, random_state=5)

In [34]:
rf.fit(x_train, y_train)

RandomForestRegressor(bootstrap=False, max_features=35, n_estimators=95,
                      random_state=5)

In [35]:
rf_pred = rf.predict(df_test)

In [36]:
pred = pd.DataFrame(rf_pred)
sub_df = pd.read_csv('../../Datasets/house-prices-advanced-regression-techniques/sample_submission.csv')
datasets = pd.concat([sub_df['Id'],pred],axis=1)

In [37]:
datasets.columns = ['Id', 'SalePrice']
datasets.to_csv('submission.csv',index=False)